# Data-Centric NLP 대회: 주제 분류 프로젝트

## Load Libraries

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

## Set Hyperparameters

In [ ]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [ ]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

## Load Tokenizer and Model

In [ ]:
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to(DEVICE)

## Define Dataset

In [ ]:
data = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
data = data[data['noise'] == 1]
data = data.dropna()

dataset_train, dataset_valid = train_test_split(data, test_size=0.3, random_state=SEED)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

In [ ]:
data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Define Metric

In [ ]:
f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')


## Train Model

In [ ]:
### for wandb setting
#os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
trainer.save_model(OUTPUT_DIR)

## Evaluate Model

In [ ]:
dataset_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [ ]:
model.eval()
preds = []

for idx, sample in tqdm(dataset_test.iterrows(), total=len(dataset_test), desc="Evaluating"):
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        preds.extend(pred)

In [ ]:
dataset_test['target'] = preds
dataset_test.to_csv(os.path.join(BASE_DIR, 'output.csv'), index=False)

# Label Correction

In [ ]:
from src.data_control.label_corrector import LabelCorrectorCleanlab

data = pd.read_csv(os.path.join(DATA_DIR, 'clean_train.csv'))
data = data.dropna(subset=["text"])
label_corrector = LabelCorrectorCleanlab(model, tokenizer, DEVICE)

# 데이터 라벨 노이즈 수정
corrected_data = label_corrector.correct(data.copy())
corrected_data.to_csv(os.path.join(BASE_DIR, 'corrected.csv'), index=False)

# 데이터 라벨 노이즈 제거
cleaned_data = label_corrector.clean(data.copy())
cleaned_data.to_csv(os.path.join(BASE_DIR, 'cleaned.csv'), index=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Cleanlab이 1117개의 라벨 이슈를 발견했습니다.
인덱스 691: 라벨 교정됨
인덱스 2159: 라벨 교정됨
인덱스 2526: 라벨 교정됨
인덱스 1103: 라벨 교정됨
인덱스 1092: 라벨 교정됨
인덱스 653: 라벨 교정됨
인덱스 1542: 라벨 교정됨
인덱스 1660: 라벨 교정됨
인덱스 121: 라벨 교정됨
인덱스 2281: 라벨 교정됨
인덱스 2276: 라벨 교정됨
인덱스 1005: 라벨 교정됨
인덱스 38: 라벨 교정됨
인덱스 1392: 라벨 교정됨
인덱스 1017: 라벨 교정됨
인덱스 662: 라벨 교정됨
인덱스 2109: 라벨 교정됨
인덱스 2211: 라벨 교정됨
인덱스 769: 라벨 교정됨
인덱스 2353: 라벨 교정됨
인덱스 2464: 라벨 교정됨
인덱스 1071: 라벨 교정됨
인덱스 2746: 라벨 교정됨
인덱스 1742: 라벨 교정됨
인덱스 2133: 라벨 교정됨
인덱스 1733: 라벨 교정됨
인덱스 549: 라벨 교정됨
인덱스 660: 라벨 교정됨
인덱스 2458: 라벨 교정됨
인덱스 61: 라벨 교정됨
인덱스 2408: 라벨 교정됨
인덱스 1107: 라벨 교정됨
인덱스 1430: 라벨 교정됨
인덱스 1659: 라벨 교정됨
인덱스 2676: 라벨 교정됨
인덱스 66: 라벨 교정됨
인덱스 2653: 라벨 교정됨
인덱스 582: 라벨 교정됨
인덱스 1576: 라벨 교정됨
인덱스 2145: 라벨 교정됨
인덱스 1481: 라벨 교정됨
인덱스 2673: 라벨 교정됨
인덱스 145: 라벨 교정됨
인덱스 1775: 라벨 교정됨
인덱스 2224: 라벨 교정됨
인덱스 2205: 라벨 교정됨
인덱스 388: 라벨 교정됨
인덱스 156: 라벨 교정됨
인덱스 304: 라벨 교정됨
인덱스 1503: 라벨 교정됨
인덱스 1232: 라벨 교정됨
인덱스 1156: 라벨 교정됨
인덱스 1309: 라벨 교정됨
인덱스 444: 라벨 교정됨
인덱스 2358: 라벨 교정됨
인덱스 1714: 라벨 교정됨
인덱스 377: 라벨 교정됨
인덱스 1646: 라벨 교정됨
인덱

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Cleanlab이 1117개의 라벨 이슈를 발견했습니다.
